# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

1. Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:  

       *Наименование вакансии  
       *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)  
       *Ссылку на саму вакансию        
       *Сайт откуда собрана вакансия  
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [67]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

In [ ]:
def get_vacancy_compensation_hh(compensation):
    vac_compensation = item.find('div', {'class':'vacancy-serp-item__compensation'}).getText()

    if vacancy_compensation != None:
        vacancy_compensation = vacancy_compensation.split(' ')
        currency_compensation = vacancy_compensation[-1]
        if vacancy_compensation[0] == 'от':
            min_compensation = ''.join(vacancy_compensation[1:-1])
            max_compensation = None
        elif vacancy_compensation[0] == 'до':
            max_compensation = ''.join(vacancy_compensation[1:-1])
            min_compensation = None
        else:
            min_compensation = vacancy_compensation[0].split('-')[0]
            max_compensation = vacancy_compensation[0].split('-')[1]
    else:
        min_compensation, max_compensation, currency_compensation = [None] * 3
            
    return min_compensation, max_compensation, currency_compensation

In [68]:
main_link_hh = 'https://hh.ru/'
job_title = 'Data+scientist'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

response = requests.get(main_link_hh + f'/search/vacancy?text={job_title}', headers=headers)

soup = bs(response.text,'lxml')

In [69]:
print(soup)

<!DOCTYPE html>
<html class="desktop" lang="ru"><!--request: 15934314349244a72fec91a1e104340a--><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Работа в Москве, поиск персонала и публикация вакансий - hh.ru</title><meta content="summary_large_image" name="twitter:card"/><meta content="hh.ru — сервис, который помогает найти работу и подобрать персонал в Москве более 19 лет! Создавайте резюме и откликайтесь на вакансии. Набирайте сотрудников и публикуйте вакансии." name="description"/><meta content="работа, вакансии, работа, поиск вакансий, резюме, работы, работу, работ, ищу работу, поиск" name="keywords"/><meta content="https://i.hh.ru/logos/openGraph/hh.ru.png?v=3" property="og:image"/><meta content="600" property="og:image:width"/><meta content="315" property="og:image:height"/><meta content="Работа в Москве, поиск персонала и публикация вакансий - hh.ru" property="og:title"/><meta content="hh.ru — 

In [70]:
vacancy_block = soup.find('div',{'class':'vacancy-serp'})

In [71]:
vacancy_list = vacancy_block.find_all('div', {'class':'vacancy-serp__vacancy'})
vacancy_list = vacancy_block.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)

In [72]:
pprint(len(vacancy_list))

50


In [73]:
vacancies = []

for vacancy in vacancy_list:
    vacancy_data = {}
    
    site_name = main_link
    vacancy_name = vacancy.find('a').getText()
    vacancy_link = vacancy.find('a')['href']
    employer = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).getText()
    location = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText()
    vacancy_compensation = vacancy.find('div', {'class':'vacancy-serp-item__compensation'}).getText()
  
    min_compensation, max_compensation, currency_compensation = get_vacancy_compensation_hh(compensation)
   
    vacancy_data['name'] = vacancy_name
    vacancy_data['link'] = vacancy_link
    vacancy_data['employer'] = employer
    vacancy_data['location'] = location
    vacancy_data['site'] = site_name
    vacancy_data['min_compensation'] = min_compensation
    vacancy_data['max_compensation'] = max_compensation
    vacancy_data['currency'] = currency_compensation
    
    vacancies.append(vacancy_data)

print(vacancies)

AttributeError: 'NoneType' object has no attribute 'getText'

In [ ]:
df = pd.DataFrame(columns=['name', 'employer', 'location', 'min_compensation', 'max_compensation',
                          'currency', 'link', 'site'])

In [ ]:
page = soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

if page is None:
    break
else:
    next_page = requests.get(main_link + page['href'], headers=header).text
    soup = bs(next_page, 'lxml')

In [ ]:
def get_vacancy_compensation_sj(compensation):
    
    vacancy_compensation = vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
    
    if vac_compensation != None:
        if vac_compensation == 'По договорённости':
            min_compensation = None
            max_compensation = None
            comp_curr = None
        elif '—' in vac_compensation:
            vac_compensation = vac_compensation.split('—')
            comp_curr = vac_compensation[1][-1]
            max_compensation = vac_compensation[1][:-1]
            min_compensation = vac_compensation[0]
        elif 'от' in vac_compensation:
            vac_compensation = vac_compensation.split()
            comp_curr = vac_compensation[-1]
            max_compensation = None
            min_compensation = ' '.join(vac_compensation[1:-1])
        elif 'до' in vac_compensation:
            vac_compensation = vac_compensation.split()
            comp_curr = vac_compensation[-1]
            max_compensation = ' '.join(vac_compensation[1:-1])
            min_compensation = None
        else:
            vac_compensation = vac_compensation.split()
            comp_curr = vac_compensation[-1]
            max_compensation = ' '.join(vac_compensation[:-1])
            min_compensation = ' '.join(vac_compensation[:-1])
    else:
        min_compensation, max_compensation, currency_compensation = [None] * 3
        
    return min_compensation, max_compensation, currency_compensation

In [74]:
main_link_sj = 'https://www.superjob.ru/'
job_title = 'Data-scientist'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

response_sj = requests.get(main_link_sj + f'/vakansii/{job_title}.html', headers=headers)

soup_sj = bs(response_sj.text,'lxml')

In [76]:
print(soup_sj)

<!DOCTYPE html>
<html class="no-js" lang="en"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><title>Data Scientist, работа Data Scientist, вакансии Data Scientist в Москве</title><meta content="Поиск работы Data Scientist в Москве. 4 вакансии Data Scientist" name="description"/><meta content="работа, вакансии, резюме, поиск работы" name="keywords"/><meta content="notranslate" name="google"/><meta content="6fa953a8f9" name="verify-admitad"/><meta content="width=device-width" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="browserconfig.xml" name="msapplication-config"/><link href="/fstc/desktop/favicon.ico" rel="shortcut icon"/><link href="/fstc/desktop/favicon.png" rel="icon" sizes="558x558" type="image/png"/><link href="/fstc/desktop/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/fstc/desktop/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/fstc/desktop/apple

In [77]:
vacancy_block_sj = soup_sj.find('div',{'class':'_3zucV'})

In [ ]:
vacancy_list_sj = vacancy_block_sj.find_all('div', {'class':'Fo44F'})
vacancy_list_sj = vacancy_block_sj.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)

In [ ]:
vacancies_sj = []

for vacancy in vacancy_list_sj:
    vacancy_data = {}
    
    site_name = main_link_sj
    vacancy_name = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText()
    vacancy_link = main_link_sj + vacancy.findParent('a')['href']
    employer = vacancy.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI'}).getText()
    location = vacancy.find('span', {'class':'_3mfro f-test-text-company-item-location _9fXTd _2JVkc _3e53o'}).findChildren(limit=2)[1].getText()
    
    vacancy_data['name'] = vacancy_name
    vacancy_data['link'] = vacancy_link
    vacancy_data['employer'] = employer
    vacancy_data['location'] = location
    vacancy_data['site'] = site_name
    vacancy_data['min_compensation'] = min_compensation
    vacancy_data['max_compensation'] = max_compensation
    vacancy_data['currency'] = currency_compensation
    
    vacancies_sj.append(vacancy_data)

print(vacancies_sj)

In [ ]:
df_sj = pd.DataFrame(columns=['name', 'employer', 'location', 'min_compensation', 'max_compensation',
                          'currency', 'link', 'site'])